Borrows from [ineTuning_phi-1_5_with_PRFT_LoRA](https://github.com/rohan-paul/LLM-FineTuning-Large-Language-Models/blob/main/FineTuning_phi-1_5_with_PRFT_LoRA.ipynb)

In [1]:
from peft import PeftModel
from transformers import AutoModelForCausalLM
import torch
from datasets import load_dataset, Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForLanguageModeling, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model
import os

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
data = load_dataset("BI55/MedText", split="train")

data_df = data.to_pandas()

data_df["text"] = data_df[["Prompt", "Completion"]].apply(lambda x: "Prompt: " + x["Prompt"] + " Completion: " + x["Completion"], axis=1)


Generating train split: 1412 examples [00:00, 78850.17 examples/s]


In [5]:
data

Dataset({
    features: ['Prompt', 'Completion'],
    num_rows: 1412
})

In [10]:
data_df["text"] = data_df.text.apply(lambda x: {"text": x})

In [11]:
data_df

,Prompt,Completion,text
0,A 50-year-old male presents with a history of ...,This patient's history of recurrent kidney sto...,{'text': 'Prompt: A 50-year-old male presents ...
1,"A 7-year-old boy presents with a fever, headac...","This child's symptoms of a red, bulging tympan...",{'text': 'Prompt: A 7-year-old boy presents wi...
2,A 35-year-old woman presents with a persistent...,While the symptoms might initially suggest ast...,{'text': 'Prompt: A 35-year-old woman presents...
3,A 50-year-old male presents with severe abdomi...,The patient's symptoms suggest an incarcerated...,{'text': 'Prompt: A 50-year-old male presents ...
4,A newborn baby presents with eye redness and a...,The infant's symptoms suggest neonatal conjunc...,{'text': 'Prompt: A newborn baby presents with...
...,...,...,...
1407,A 55-year-old male with a history of chronic o...,While this patient's symptoms could be due to ...,{'text': 'Prompt: A 55-year-old male with a hi...
1408,Can diet and lifestyle changes help manage vit...,While there is no specific diet or lifestyle m...,{'text': 'Prompt: Can diet and lifestyle chang...
1409,A 50-year-old female presents with right shoul...,This patient's shoulder and arm pain following...,{'text': 'Prompt: A 50-year-old female present...
1410,A 60-year-old female with high cholesterol lev...,In addition to a diet low in saturated fats an...,{'text': 'Prompt: A 60-year-old female with hi...


In [12]:
datasets_folder = "../../src/text-generation-webui/training/datasets/huggingface/BI55-MedText"
data_df["text"].to_json(f"{datasets_folder}/completion_training_text.json", orient="records", lines=True)